In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import xarray as xr
import cmocean.cm as cmo 
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import matplotlib as mpl
from scipy.spatial import Delaunay


from matplotlib.colors import LinearSegmentedColormap
import gsw
from mpl_toolkits.axes_grid1 import make_axes_locatable
#from Useful import *
from matplotlib.collections import PolyCollection
from scipy.interpolate import griddata
from scipy.spatial import KDTree
import imageio
import os
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [4]:
path1 = '../../../results_20251126_002/'      # threshold_thickness calving front 

In [5]:
i = 2
Op1 = xr.open_dataset(path1 + f'main_output_ANT_{i:05d}.nc')
Hi1 = Op1['Hi']

Va1 = Op1['V'] 
V1 = Va1.T
Tria1 = Op1['Tri'] - 1  #Vertex boundary index 
#-> more info in Tijn's paper C. J. Berends et al.: The Utrecht Finite Volume Ice-Sheet Model (UFEMISM version) 2.0 – Part I appendic A
Tri1 = Tria1.T
triang1 = tri.Triangulation(V1[:, 0], V1[:, 1], Tri1)
Cf1= Op1['mask_margin']
Gr1 = Op1['mask_grounded_ice']
time = Op1['time']
a = -1

colorbar_kwargs = {
    'aspect': 10,
    'pad': 0.01,
    'shrink': 0.75, 
}
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(16, 8))

# Left: Ice thickness
c0 = ax0.tripcolor(triang1, Hi1[a], cmap=cmo.balance, vmin=0, vmax=600)
ax0.tricontourf(triang1, Gr1[a].astype(float), 
                levels=[0.5, 1.5], colors='none', 
                hatches=['//'], alpha=0.15) 
cbar0 = fig.colorbar(c0, ax=ax0, **colorbar_kwargs)
cbar0.set_label("Ice Thickness [m]", fontsize=14)
ax0.set_title(f'Ice thickness [m] at t = {time[a].values} years', fontsize=15)
ax0.set_xlabel('X [km]', fontsize=13)
ax0.set_ylabel('Y [km]', fontsize=13)

# Right: Mask (calving front / floating)
c1 = ax1.tripcolor(triang1, Cf1[a], cmap=cmo.balance)
cbar1 = fig.colorbar(c1, ax=ax1, **colorbar_kwargs)
cbar1.set_label("Mask value", fontsize=14)
ax1.set_title(f'Calving front / floating mask at t = {time[a].values} years', fontsize=15)
ax1.set_xlabel('X [km]', fontsize=13)
ax1.set_ylabel('Y [km]', fontsize=13)

# Common formatting
n = 800000
xt = np.arange(-n, n + 1, 200000)
xtl = np.arange(-n // 1000, n // 1000 + 1, 200)
for ax in (ax0, ax1):
    ax.set_xticks(xt)
    ax.set_xticklabels(xtl, fontsize=12)
    ax.set_yticks(xt)
    ax.set_yticklabels(xtl, fontsize=12)
    ax.set_xlim(-n, n)
    ax.set_ylim(-n, n)
    ax.set_aspect('equal')


plt.tight_layout()
plt.show()


KeyError: "No variable named 'mask_margin'. Variables on the dataset include ['xmin', 'xmax', 'ymin', 'ymax', 'tol_dist', ..., 'mask_floating_ice', 'time', 'zeta', 'month', 'depth']"

In [ ]:


path1 = '../../../results_20251117_019/'  # Update this path
temp_files = []
images = []
colorbar_kwargs = {
    'aspect': 10,
    'pad': 0.01,
    'shrink': 0.785, 
}

# Loop through all 4 files
for i in range(1, 5):  # Files 1 through 4
    print(f"Processing file {i}...")
    
    try:
        Op1 = xr.open_dataset(path1 + f'main_output_ANT_{i:05d}.nc')
        
        # Extract data for this file
        Hi1 = Op1['Hi']
        Va1 = Op1['V'] 
        V1 = Va1.T
        Tria1 = Op1['Tri'] - 1
        Tri1 = Tria1.T
        triang1 = tri.Triangulation(V1[:, 0], V1[:, 1], Tri1)
        Cf1 = Op1['mask_margin']
        Gr1 = Op1['mask_grounded_ice']
        time = Op1['time'].values

        # Process each time step in this file
        for u in range(len(Op1['time'])):
            print(f"  Time step {u+1}/{len(Op1['time'])}: t = {time[u]:.1f} years")
            
            fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(16, 8))

            # Left: Ice thickness
            c0 = ax0.tripcolor(triang1, Hi1[u], cmap=cmo.balance, vmin=0, vmax=600)
            ax0.tricontourf(triang1, Gr1[u].astype(float), 
                levels=[0.5, 1.5], colors='none', 
                hatches=['//'], alpha=0.15) 
            cbar0 = fig.colorbar(c0, ax=ax0, **colorbar_kwargs)
            cbar0.set_label("Ice Thickness [m]", fontsize=14)
            ax0.set_title('Ice thickness [m]', fontsize=15)
            ax0.set_xlabel('X [km]', fontsize=13)
            ax0.set_ylabel('Y [km]', fontsize=13)

            # Right: Mask (calving front / floating)
            c1 = ax1.tripcolor(triang1, Cf1[u], cmap=cmo.balance)
            cbar1 = fig.colorbar(c1, ax=ax1, **colorbar_kwargs)
            cbar1.set_label("Mask value", fontsize=14)
            ax1.set_title('Calving front', fontsize=15)
            ax1.set_xlabel('X [km]', fontsize=13)
            ax1.set_ylabel('Y [km]', fontsize=13)

            # Common formatting
            n = 800000
            xt = np.arange(-n, n + 1, 200000)
            xtl = np.arange(-n // 1000, n // 1000 + 1, 200)
            for ax in (ax0, ax1):
                ax.set_xticks(xt)
                ax.set_xticklabels(xtl, fontsize=12)
                ax.set_yticks(xt)
                ax.set_yticklabels(xtl, fontsize=12)
                ax.set_xlim(-n, n)
                ax.set_ylim(-n, n)
                ax.set_aspect('equal')
            
            fig.suptitle(f'Time = {time[u]:.1f} years (File {i})', fontsize=16, y=0.9)
            plt.tight_layout()

            # Save frame
            filename = f'TEMP_file{i}_time{time[u]:.1f}.png'
            plt.savefig(filename, bbox_inches='tight', dpi=300) 
            temp_files.append(filename) 
            plt.close()
            images.append(imageio.imread(filename))
            
        # Close the dataset when done with this file
        Op1.close()
        
    except FileNotFoundError:
        print(f"File {i} not found, skipping...")
        continue
    except Exception as e:
        print(f"Error processing file {i}: {e}")
        continue

# Create GIF if we have images
if images:
    print(f"Creating GIF with {len(images)} frames...")
    imageio.mimsave('Calving_animation.gif', images, fps=3, loop=0)
    print("GIF saved as 'Calving_animation.gif'")
else:
    print("No images were created!")

# Clean up temporary files
print("Cleaning up temporary files...")
for filename in temp_files:
    try:
        os.remove(filename)
        print(f"  Deleted: {filename}")
    except OSError as e:
        print(f"  Error deleting {filename}: {e}")

print("Process completed!")

Processing file 1...
  Time step 1/1: t = 0.0 years


/var/folders/qd/ggy847q57w1d6rr22f838tvm0000gn/T/ipykernel_7256/2480175062.py:74: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))


Processing file 2...
  Time step 1/100: t = 0.1 years


/var/folders/qd/ggy847q57w1d6rr22f838tvm0000gn/T/ipykernel_7256/2480175062.py:74: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))


  Time step 2/100: t = 0.2 years


...

  Time step 48/49: t = 24.8 years


/var/folders/qd/ggy847q57w1d6rr22f838tvm0000gn/T/ipykernel_7256/2480175062.py:74: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))


  Time step 49/49: t = 24.9 years


/var/folders/qd/ggy847q57w1d6rr22f838tvm0000gn/T/ipykernel_7256/2480175062.py:74: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))


Creating GIF with 250 frames...
GIF saved as 'Calving_animation.gif'
Cleaning up temporary files...
  Deleted: TEMP_file1_time0.0.png
  Deleted: TEMP_file2_time0.1.png
  Deleted: TEMP_file2_time0.2.png
  Deleted: TEMP_file2_time0.3.png
  Deleted: TEMP_file2_time0.4.png
  Deleted: TEMP_file2_time0.5.png
  Deleted: TEMP_file2_time0.6.png
  Deleted: TEMP_file2_time0.7.png
  Deleted: TEMP_file2_time0.8.png
  Deleted: TEMP_file2_time0.9.png
  Deleted: TEMP_file2_time1.0.png
  Deleted: TEMP_file2_time1.1.png
  Deleted: TEMP_file2_time1.2.png
  Deleted: TEMP_file2_time1.3.png
  Deleted: TEMP_file2_time1.4.png
  Deleted: TEMP_file2_time1.5.png
  Deleted: TEMP_file2_time1.6.png
  Deleted: TEMP_file2_time1.7.png
  Deleted: TEMP_file2_time1.8.png
  Deleted: TEMP_file2_time1.9.png
  Deleted: TEMP_file2_time2.0.png
  Deleted: TEMP_file2_time2.1.png
  Deleted: TEMP_file2_time2.2.png
  Deleted: TEMP_file2_time2.3.png
  Deleted: TEMP_file2_time2.4.png
  Deleted: TEMP_file2_time2.5.png
  Deleted: TEMP_

In [6]:
i = 1
Op1 = xr.open_dataset(path1 + f'main_output_ANT_{i:05d}.nc')
Hi1 = Op1['Hi']

Va1 = Op1['V'] 
V1 = Va1.T
Tria1 = Op1['Tri'] - 1  #Vertex boundary index 
#-> more info in Tijn's paper C. J. Berends et al.: The Utrecht Finite Volume Ice-Sheet Model (UFEMISM version) 2.0 – Part I appendic A
Tri1 = Tria1.T
triang1 = tri.Triangulation(V1[:, 0], V1[:, 1], Tri1)
Cf1= Op1['mask_cf_fl']

a = 0

colorbar_kwargs = {
    'aspect': 10,
    'pad': 0.01,
    'shrink': 0.75, 
}
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(16, 8))

# Left: Ice thickness
c0 = ax0.tripcolor(triang1, Hi1[a], cmap=cmo.balance, vmin=0, vmax=600)
cbar0 = fig.colorbar(c0, ax=ax0, **colorbar_kwargs)
cbar0.set_label("Ice Thickness [m]", fontsize=14)
ax0.set_title('Ice thickness [m] at t = 0 years', fontsize=15)
ax0.set_xlabel('X [km]', fontsize=13)
ax0.set_ylabel('Y [km]', fontsize=13)

# Right: Mask (calving front / floating)
c1 = ax1.tripcolor(triang1, Cf1[a], cmap=cmo.balance)
cbar1 = fig.colorbar(c1, ax=ax1, **colorbar_kwargs)
cbar1.set_label("Mask value", fontsize=14)
ax1.set_title('Calving front / floating mask (t = 0 years)', fontsize=15)
ax1.set_xlabel('X [km]', fontsize=13)
ax1.set_ylabel('Y [km]', fontsize=13)

# Common formatting
n = 800000
xt = np.arange(-n, n + 1, 200000)
xtl = np.arange(-n // 1000, n // 1000 + 1, 200)
for ax in (ax0, ax1):
    ax.set_xticks(xt)
    ax.set_xticklabels(xtl, fontsize=12)
    ax.set_yticks(xt)
    ax.set_yticklabels(xtl, fontsize=12)
    ax.set_xlim(-n, n)
    ax.set_ylim(-n, n)
    ax.set_aspect('equal')


plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/McQua002/Desktop/UFEMISM.nosync/results_20251117_002/main_output_ANT_00001.nc'